In [1]:
import getpass
import requests as re
import json
from datetime import datetime
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
#staging_server = "https://staging-api.globalforestwatch.org"
staging_server = "https://staging-api.resourcewatch.org"
prod_server = "https://api.resourcewatch.org"

In [3]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
def auth(env='prod'):
    serverUrl = {
        'prod': prod_server,
        'staging': staging_server
    }
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}{env}{bcolors.ENDC}')
    with re.Session() as s:
        headers = {'Content-Type': 'application/json'}
        payload = json.dumps({ 'email': f'{input(f"Email: ")}',
                               'password': f'{getpass.getpass(prompt="Password: ")}'})
        response = s.post(f'{serverUrl[env]}/auth/login',  headers = headers,  data = payload)
        response.raise_for_status()
        print(f'{bcolors.OKGREEN}Successfully logged into {env}{bcolors.ENDC}')
    return response.json().get('data').get('token')

In [5]:
token = {
    'staging': auth('staging'),
    'prod':auth('prod')
}

You are login into staging


Email:  tamara.huete@vizzuality.com
Password:  ···········


Successfully logged into staging
You are login into prod


Email:  tamara.huete@vizzuality.com
Password:  ········


Successfully logged into prod


In [6]:
def setTokenHeader(env, token=token):
    '''
    set up the token
    '''
    return {'Authorization':f'Bearer {token[env]}', 
            'Content-Type': 'application/json'}

def logResponseErrors(status_code, response = None, url = None, body = None):
    '''
    log errors in http calls
    '''
    if status_code !=200:
        logging.error('response: ')
        logging.error(response)
        logging.error(response.text) if response else None
        logging.error(response.json()) if response else None
        logging.error('url: ')
        logging.error(url) if url else None
        logging.error('body: ')
        logging.error(json.dumps(body)) if body else None
        
def postAssets(url, body, headers, payloads = None):
    '''
    create asset operation
    '''    
    response = re.post(url, params = payloads, data=json.dumps(body), headers = headers)
    
    logResponseErrors(response.status_code, response, url, body)
    
    response.raise_for_status()
    
    return response.json()

def createWidget(datasetId, body, toEnv = 'prod', destinationWidgetId = None):
    '''
    Copy the widget from one env to the other
    '''
    
    serverUrl = {
        'prod': prod_server,
        'staging': staging_server
    }
    
    headers = setTokenHeader(toEnv)
    url = f'{serverUrl[toEnv]}/v1/dataset/{datasetId}/widget'
    
    if destinationWidgetId:
        url = f'{url}/{destinationWidgetId}'
    
    return postAssets(url, body, headers)

In [7]:
widgetBody = json.loads('''
    {
    "name": "Reduction in Life Expectancy Map",
    "description": "",
    "source": "",
    "authors": "",
    "application": ["rw"],
    "verified": false,
    "default": false,
    "protected": false,
    "defaultEditableWidget": false,
    "published": true,
    "freeze": false,
    "env": "production",
    "queryUrl": "",
    "widgetConfig": {
        "lng": 4.504394531250001,
        "lat": -28.497660832963472,
        "zoom": 4,
        "type": "map",
        "paramsConfig": {
            "embed": {
                "src": ""
                },
            "band": null,
            "areaIntersection": null,
            "filters": [],
            "chartType": null,
            "aggregateFunction": null,
            "orderBy": null,
            "size": null,
            "color": null,
            "category": null,
            "value": null,
            "limit": 500,
            "visualizationType": "map",
            "layer_id": "b0a8f63f-e3a3-4599-abd8-19120edd98b1",
            "layer": "b0a8f63f-e3a3-4599-abd8-19120edd98b1",
            "layers": [
                "2a694289-fec9-4bfe-a6d2-56c3864ec349",
                "155968b5-7c59-4065-9e3a-0a81d52d50de",
                "00e688a8-086e-4206-9ea7-b47afca6913c",
                "7890b4f2-502a-408b-8614-3f00e0b53ec2"
                  ]
                }
            }
  }
''')

## Staging Layers OW

# Catchment Areas and Land cover

## Sediment Pressure

In [15]:
DatasetId = 'ae3ffd82-19a8-4d7c-a974-cc6d7b512f86'
layers = ['c4c7b8e1-1d74-40fe-840c-e8aa185a4461', '0f27b5ee-21e0-48d4-81ba-f8d33d773514', '9b9fe210-f99d-452f-b5d8-9bae0b1a5ec2']
widgetBody['name'] = "Sediment Pressure"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'ac8fb011-b003-466e-ab5e-1ccd417bbc1d',
  'type': 'widget',
  'attributes': {'name': 'Sediment Pressure',
   'dataset': 'ae3ffd82-19a8-4d7c-a974-cc6d7b512f86',
   'slug': 'Sediment-Pressure',
   'userId': '57a0aa1071e394dd32ffe137',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': 'c4c7b8e1-1d74-40fe-840c-e8a

## Water Quality Pressure

In [16]:
DatasetId = 'f7bbc39e-81ab-4d91-be37-c12e33cd8436'
layers = ['0f27b5ee-21e0-48d4-81ba-f8d33d773514','c4c7b8e1-1d74-40fe-840c-e8aa185a4461','abefb374-3355-4110-9c03-c7031f0b61b4','fe6315f7-e208-441d-9193-9dee6499b349']
widgetBody['name'] = "Water Quality Pressure"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': '913ab9b7-5e5c-4830-a4c8-5e3b2d45db77',
  'type': 'widget',
  'attributes': {'name': 'Water Quality Pressure',
   'dataset': 'f7bbc39e-81ab-4d91-be37-c12e33cd8436',
   'slug': 'Water-Quality-Pressure',
   'userId': '57a0aa1071e394dd32ffe137',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': '8339b4a5-b99f-4d6

# River Water Quality

## Sediment Pressure (type map slider)

In [30]:
params_config = {'embed': {'src': ''},
 'band': None,
 'areaIntersection': None,
 'filters': [],
 'chartType': None,
 'aggregateFunction': None,
 'orderBy': None,
 'size': None,
 'color': None,
 'category': None,
 'value': None,
 'limit': 500,
 'visualizationType': 'map-slide',
 'layer_id': '0f27b5ee-21e0-48d4-81ba-f8d33d773514',
 'layer': '0f27b5ee-21e0-48d4-81ba-f8d33d773514',
 'layerParams': {
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514': {'opacity': 0.01},
  'f37d8246-ddb6-4dbb-a9db-715fa889a88b': {'opacity': 1},
  '46f41568-dd10-44e8-861a-8a552a40339b': {'opacity': 1}},
 'layersLeft': [
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514',
  '46f41568-dd10-44e8-861a-8a552a40339b'],
 'layersRight': [
  'f37d8246-ddb6-4dbb-a9db-715fa889a88b',
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514'],
 'mask': {'account': 'wri-rw',
  'body': {'layers': [{'type': 'mapnik',
     'options': {'sql': "SELECT cartodb_id, the_geom_webmercator, st_intersects(the_geom, (select the_geom from gadm36_0 where {{geostore_env}} = '{{geostore_id}}')) as intersect FROM wat_068_rw0_watersheds_edit WHERE level = 3"}}],
   'vectorLayers': [{'filter': ['in', 'intersect', False],
     'paint': {'fill-color': '#616161',
      'fill-opacity': 1,
      'fill-outline-color': '#616161'},
     'source-layer': 'layer0',
     'type': 'fill'},
    {'filter': ['in', 'intersect', True],
     'paint': {'line-color': '#ffa500', 'line-width': 1.5},
     'source-layer': 'layer0',
     'type': 'line'},
    {'filter': ['in', 'intersect', True],
     'paint': {'line-color': '#616161', 'line-width': 0.7},
     'source-layer': 'layer0',
     'type': 'line'}],
   'layerType': 'vector'}}}

In [32]:
DatasetId = 'e16d6b2d-7084-4891-878c-ae47fc49da52'
widgetBody['name'] = "River Water Quality - Sedimet pressure Map"
widgetBody['widgetConfig']['paramsConfig'] = params_config
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'ab52220e-768f-4d02-93d3-feca04ff9381',
  'type': 'widget',
  'attributes': {'name': 'River Water Quality - Sedimet pressure Map',
   'dataset': 'e16d6b2d-7084-4891-878c-ae47fc49da52',
   'slug': 'River-Water-Quality-Sedimet-pressure-Map',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map

## Water Quality Pressure (type map slider)

In [33]:
DatasetId = 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc'
layersRight = [
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514',
  'abefb374-3355-4110-9c03-c7031f0b61b4']
layersLeft = [
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514',
  'f065c648-7579-4034-9d6f-dd1fcbd1b550']
layerParams = {
  '0f27b5ee-21e0-48d4-81ba-f8d33d773514': {'opacity': 0.01},
  'abefb374-3355-4110-9c03-c7031f0b61b4': {'opacity': 1},
  'f065c648-7579-4034-9d6f-dd1fcbd1b550': {'opacity': 1}}
widgetBody['name'] = "River Water Quality - Water quality pressure Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'c22b02ea-84e1-4325-8125-762188ace24a',
  'type': 'widget',
  'attributes': {'name': 'River Water Quality - Water quality pressure Map',
   'dataset': 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc',
   'slug': 'River-Water-Quality-Water-quality-pressure-Map',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizatio

# Ecosystems and Pressures

In [ ]:
DatasetId = ''
layers = ['','','']
widgetBody['name'] = "Reduction in Life Expectancy Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

# Values of Goods and Services

### Tourism

In [9]:
DatasetId = '6d377aa4-245f-425e-a3d8-e5014e3d8edf'
layers = ['c4c7b8e1-1d74-40fe-840c-e8aa185a4461','abefb374-3355-4110-9c03-c7031f0b61b4','d97de866-6e33-4893-ae1d-fa05703925fe']
widgetBody['name'] = "Tourism"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': '2a1db525-7352-4c88-a17e-5705e8860888',
  'type': 'widget',
  'attributes': {'name': 'Tourism',
   'dataset': '6d377aa4-245f-425e-a3d8-e5014e3d8edf',
   'slug': 'Tourism',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': 'c4c7b8e1-1d74-40fe-840c-e8aa185a4461',
     'la

### Fisheries (mang)

In [11]:
DatasetId = '10c26b23-b7cf-48fc-b5bf-dc3284fb8b1e'
layers = ['52c01177-2aeb-400c-9171-eefef5529c66','c4c7b8e1-1d74-40fe-840c-e8aa185a4461','d0569782-ca95-4821-853a-f8e37f77eec8']
widgetBody['name'] = "Fisheries (mang)"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'b191aed0-1f56-4273-b5f2-dd264f951f3d',
  'type': 'widget',
  'attributes': {'name': 'Fisheries (mang)',
   'dataset': '10c26b23-b7cf-48fc-b5bf-dc3284fb8b1e',
   'slug': 'Fisheries-mang',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': '52c01177-2aeb-400c-9171-eefef55

### Fisheries (coral)

In [12]:
DatasetId = 'e935f438-b508-40af-b13c-e3b7f6d8651c'
layers = ['2d5b79ae-32f7-4bef-ad38-ae9dcc6188db','52c01177-2aeb-400c-9171-eefef5529c66',
          'c4c7b8e1-1d74-40fe-840c-e8aa185a4461','d0569782-ca95-4821-853a-f8e37f77eec8']
widgetBody['name'] = "Fisheries (coral)"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': '0d216610-b4d2-4c4e-a5d2-14f83112a2ee',
  'type': 'widget',
  'attributes': {'name': 'Fisheries (coral)',
   'dataset': 'e935f438-b508-40af-b13c-e3b7f6d8651c',
   'slug': 'Fisheries-coral',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': '2d5b79ae-32f7-4bef-ad38-ae9dc

### Shoreline prot

In [14]:
DatasetId = 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc'
layers = ['abefb374-3355-4110-9c03-c7031f0b61b4','52c01177-2aeb-400c-9171-eefef5529c66']
widgetBody['name'] = "Shoreline prot"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'd6331a40-57cd-40cf-9953-c966b6cb0e30',
  'type': 'widget',
  'attributes': {'name': 'Shoreline prot',
   'dataset': 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc',
   'slug': 'Shoreline-prot',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': 'abefb374-3355-4110-9c03-c7031f0b6

### Blue Carbon

In [15]:
DatasetId = 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc'
layers = ['67b7b13f-f068-45c1-8f45-5c4147a38e69','abefb374-3355-4110-9c03-c7031f0b61b4']
widgetBody['name'] = "Blue Carbon"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': '2a054b7b-c42e-43d9-b6ad-f31fc67a82d4',
  'type': 'widget',
  'attributes': {'name': 'Blue Carbon',
   'dataset': 'cb75c884-7e0a-436d-89c8-c27bf6fb31dc',
   'slug': 'Blue-Carbon',
   'userId': '60d1fc0d947dc2001bb5135d',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': '67b7b13f-f068-45c1-8f45-5c4147a38e69',
